In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm, norm
import pandas as pd
%matplotlib inline
import numpy.random as npr

# Step 1

Q1 : Team Member A

In [2]:
# Black-Scholes model in Python
import numpy as np
import scipy.stats as ss

# Data for input in Black-Scholes formula:

T = 0.25  # supposed in years. It is not the maturity, but the time to maturity
S = 100.0
K = 100.0
r = 0.05
vol = 0.20  # supposing it is annual

# dividend yield assumed to be 0

# Compute d1 and d2
d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
d2 = d1 - vol * np.sqrt(T)

for option_type in ["C", "P"]:
    print(option_type)
    if option_type == "C":
        Opt_Price = S * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
        Delta = ss.norm.cdf(d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) - r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(d2)
        Rho = K * T * np.exp(-r * T) * ss.norm.cdf(d2)
        print("Option price = {}".format(round(Opt_Price,2)))
        print("Delta = {}".format(round(Delta,2)))
        print("Gamma = {}".format(round(Gamma,2)))
        print("Vega = {}".format(round(Vega,2)))
        print("Theta = {}".format(round(Theta,2)))
        print("Rho = {}".format(round(Rho,2)))
    else:
        Opt_Price = K * np.exp(-r * T) * ss.norm.cdf(-d2) - S * ss.norm.cdf(-d1)
        Delta = -ss.norm.cdf(-d1)
        Gamma = ss.norm.pdf(d1) / (S * vol * np.sqrt(T))
        Vega = S * ss.norm.pdf(d1) * np.sqrt(T)
        Theta = -(S * ss.norm.pdf(d1) * vol) / (2 * np.sqrt(T)) + r * K * np.exp(
            -r * T
        ) * ss.norm.cdf(-d2)
        Rho = -K * T * np.exp(-r * T) * ss.norm.cdf(-d2)
        print("Option price = {}".format(round(Opt_Price,2)))
        print("Delta = {}".format(round(Delta,2)))
        print("Gamma = {}".format(round(Gamma,2)))
        print("Vega = {}".format(round(Vega,2)))
        print("Theta = {}".format(round(Theta,2)))
        print("Rho = {}".format(round(Rho,2)))

C
Option price = 4.61
Delta = 0.57
Gamma = 0.04
Vega = 19.64
Theta = -10.47
Rho = 13.08
P
Option price = 3.37
Delta = -0.43
Gamma = 0.04
Vega = 19.64
Theta = -5.54
Rho = -11.61


In [3]:
import scipy.stats as ss
import numpy as np

# Define the Black-Scholes function
def black_scholes(S, K, T, r, sigma, option_type='call'):
    # S: spot price
    # K: strike price
    # T: time to maturity
    # r: interest rate
    # sigma: volatility of underlying asset
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = (S * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2))
    elif option_type == 'put':
        option_price = (K * np.exp(-r * T) * ss.norm.cdf(-d2) - S * ss.norm.cdf(-d1))
    return round(option_price,2)

# Given parameters
S = 100     # Underlying asset price
K = 100     # Option strike price
T = 0.25       # Time to maturity in years
r = 0.05    # Risk-free rate
sigma = 0.2 # Volatility (initial)

# Calculate prices with initial volatility
call_price_initial = black_scholes(S, K, T, r, sigma, 'call')
put_price_initial = black_scholes(S, K, T, r, sigma, 'put')

# New volatility after 5% increase
sigma_new = 0.25

# Calculate prices with increased volatility
call_price_new = black_scholes(S, K, T, r, sigma_new, 'call')
put_price_new = black_scholes(S, K, T, r, sigma_new, 'put')

# Calculate the change in prices
call_price_change = call_price_new - call_price_initial
put_price_change = put_price_new - put_price_initial

(call_price_initial, call_price_new, round(call_price_change,2)), (put_price_initial, put_price_new, round(put_price_change,2))


((4.61, 5.6, 0.99), (3.37, 4.36, 0.99))

Q2: Team Member B

In [4]:
import numpy as np

def monte_carlo_simulation(S0, r, sigma, T, N):
    dt = 1 / 365  # Daily time-step
    ST = np.zeros(N)

    # Generation of random normal variables for each simulation
    z = np.random.normal(0, 1, N)

    # Simulate stock price paths
    for i in range(N):
        ST[i] = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z[i])

    call_payoffs = np.maximum(ST - S0, 0)
    put_payoffs = np.maximum(S0 - ST, 0)

    call_price = np.exp(-r * T) * np.mean(call_payoffs)
    put_price = np.exp(-r * T) * np.mean(put_payoffs)

    # Delta computation
    S_increment = 0.01
    delta_call = (np.exp(-r * T) * np.mean(np.maximum(ST + S_increment - S0, 0)) - call_price) / S_increment
    delta_put = (np.exp(-r * T) * np.mean(np.maximum(S0 - (ST + S_increment), 0)) - put_price) / S_increment

    # Vega computation
    new_sigma = sigma + 0.05
    ST_new = S0 * np.exp((r - 0.5 * new_sigma ** 2) * T + new_sigma * np.sqrt(T) * z)
    call_price_new = np.exp(-r * T) * np.mean(np.maximum(ST_new - S0, 0))
    put_price_new = np.exp(-r * T) * np.mean(np.maximum(S0 - ST_new, 0))
    vega_call = call_price_new - call_price
    vega_put = put_price_new - put_price

    return call_price, put_price, delta_call, delta_put, vega_call, vega_put


S0 = 100  # Initial stock price
r = 0.05  # Risk-free interest rate
sigma = 0.20  # Volatility
T = 0.25  # Time to maturity in years (3 months)
N = 100000  # Number of simulations

# Perform Monte Carlo simulation
call_price, put_price, delta_call, delta_put, vega_call, vega_put = monte_carlo_simulation(S0, r, sigma, T, N)
print("European Call Option Price:", round(call_price, 2))
print("European Put Option Price:", round(put_price, 2))

European Call Option Price: 4.6
European Put Option Price: 3.4


In [5]:
print("Delta for European Call Option:", round(delta_call, 2))
print("Delta for European Put Option:", round(delta_put, 2))

Delta for European Call Option: 0.52
Delta for European Put Option: -0.47


In [6]:
print("Vega for European Call Option:", round(vega_call, 2))
print("Vega for European Put Option:", round(vega_put, 2))

Vega for European Call Option: 0.98
Vega for European Put Option: 0.99


In [7]:

S0 = 100
r = 0.05
sigma = 0.20
T = 90 / 365
N = 100000

call_price_20, put_price_20, delta_call_20, delta_put_20, vega_call_20, vega_put_20 = monte_carlo_simulation(S0, r, sigma, T, N)

sigma2 = 0.25  # New volatility (25%)

# Recomputation option prices with increased volatility
call_price_25, put_price_25, delta_call_25, delta_put_25, vega_call_25, vega_put_25 = monte_carlo_simulation(S0, r, sigma2, T, N)

# Calculation of the change in option prices due to the increase in volatility
call_price_change = call_price_25 - call_price_20
put_price_change = put_price_25 - put_price_20

# Output
print("Change in Call Option Price due to 5% increase in volatility:", round(call_price_change, 2))
print("Change in Put Option Price due to 5% increase in volatility:", round(put_price_change, 2))


Change in Call Option Price due to 5% increase in volatility: 0.95
Change in Put Option Price due to 5% increase in volatility: 0.99


Q3: Team member C

In [8]:
import numpy as np

#Checking that put-call parity holds true for the option priced using the Monte-Carlo Simulation Method
def check_put_call_parity(call_price, put_price, S0, K, r, T): #function to check that put-call parity holds
    # Calculate left side (LHS)
    left_side = call_price - put_price

    # Calculate right side (RHS)
    right_side = S0 - K * np.exp(-r * T)

    # Calculate theoretical option prices
    theoretical_call_price = np.round(put_price + S0 - K * np.exp(-r * T),2)
    theoretical_put_price = np.round(call_price - S0 + K * np.exp(-r * T),2)

    # Check put-call parity
    parity_holds = np.isclose(left_side, right_side, atol=1e-2)

    # Store the results in a dictionary
    result = {"strike": K, "parity_holds": parity_holds, "theoretical_call_price": theoretical_call_price, "theoretical_put_price": theoretical_put_price}

    return result

# Example usage
call_price = 4.61 #call price determined from Question 1
put_price = 3.37 #put price determined from Question 1
S0 = 100
K = 100
r = 0.05
T = 0.25

parity_result = check_put_call_parity(call_price, put_price, S0, K, r, T) #Check if put-call parity holds for Question
print(parity_result)

{'strike': 100, 'parity_holds': True, 'theoretical_call_price': 4.61, 'theoretical_put_price': 3.37}


Q3(b) : Team member C

In [9]:
#Checking that put-call parity holds true for the option priced using the Black-Scholes Method

def check_put_call_parity(call_price, put_price, S0, K, r, T): #function to check that put-call parity holds
    # Calculate left side (LHS)
    left_side = call_price - put_price

    # Calculate right side (RHS)
    right_side = S0 - K * np.exp(-r * T)

    # Calculate theoretical option prices
    theoretical_call_price = np.round(put_price + S0 - K * np.exp(-r * T),2)
    theoretical_put_price = np.round(call_price - S0 + K * np.exp(-r * T),2)

    # Check put-call parity
    parity_holds = np.isclose(left_side, right_side, atol=1e-2)

    # Store the results in a dictionary
    result = {"strike": K, "parity_holds": parity_holds, "theoretical_call_price": theoretical_call_price, "theoretical_put_price": theoretical_put_price}

    return result

# Example usage
call_price = 4.62 #call price determined from Question 1
put_price = 3.38 #put price determined from Question 1
S0 = 100
K = 100
r = 0.05
T = 0.25

parity_result = check_put_call_parity(call_price, put_price, S0, K, r, T) #Check if put-call parity holds for Question
print(parity_result)

{'strike': 100, 'parity_holds': True, 'theoretical_call_price': 4.62, 'theoretical_put_price': 3.38}


# Step 2

Q4: Team Member A :

In [ ]:
import numpy as np

# Parameters for the option
S0 = 100       # initial stock price
K = 100        # strike price
T = 0.25       # time to maturity in years
r = 0.05       # risk-free rate
sigma = 0.20   # volatility
N = 100000     # number of Monte Carlo simulations

def monte_carlo_simulation_american_call(S0, K, r, sigma, T, N):
    dt = 1 / 365  # Daily time-step
    ST = np.zeros(N)

    # Generate random normal variables for each simulation
    z = np.random.normal(0, 1, N)

    # Simulate stock price paths
    for i in range(N):
        S = S0
        for t in range(int(T * 365)):
            S += r * S * dt + sigma * S * np.sqrt(dt) * z[i]
            S = max(S, 0)  # Ensure stock price doesn't go negative
        ST[i] = S

    # Compute option payoffs
    call_payoffs = np.maximum(ST - K, 0)

    # Discounted option price
    call_price = np.exp(-r * T) * np.mean(call_payoffs)

    # Delta computation
    S_increment = 0.01  # Small increment in stock price
    delta_call = (np.exp(-r * T) * np.mean(np.maximum((ST + S_increment) - K, 0)) - call_price) / S_increment

    # Vega computation
    new_sigma = sigma + 0.05
    ST_new = np.zeros(N)
    for i in range(N):
        S = S0
        for t in range(int(T * 365)):
            S += r * S * dt + new_sigma * S * np.sqrt(dt) * z[i]
            S is max(S, 0)  # Ensure stock price doesn't go negative
        ST_new[i] = S
    call_price_new = np.exp(-r * T) * np.mean(np.maximum(ST_new - K, 0))
    vega_call = (call_price_new - call_price) / 0.05  # Adjusted for a 5% increase

    return np.round(call_price,2), np.round(delta_call,2), np.round(vega_call,2)

# Perform Monte Carlo simulation for American call option
call_price, delta_call, vega_call = monte_carlo_simulation_american_call(S0, K, r, sigma, T, N)

(call_price, delta_call, vega_call)

(80.5, 0.5, 940.35)

Q5: Team Member B

In [ ]:
import numpy as np

def monte_carlo_simulation_american_put(S0, K, r, sigma, T, N):
    dt = 1 / 365  # Daily time-step
    ST = np.zeros(N)

    # Generate random normal variables for each simulation
    z = np.random.normal(0, 1, N)

    # Simulate stock price paths
    for i in range(N):
        S = S0
        for t in range(int(T * 365)):
            S += r * S * dt + sigma * S * np.sqrt(dt) * z[i]
            S = max(S, 0)  # Ensure stock price doesn't go negative
        ST[i] = S

    # Compute option payoffs
    put_payoffs = np.maximum(K - ST, 0)

    # Discounted option price
    put_price = np.exp(-r * T) * np.mean(put_payoffs)

    # Delta computation
    S_increment = 0.01  # Small increment in stock price
    delta_put = (np.exp(-r * T) * np.mean(np.maximum(K - (ST + S_increment), 0)) - put_price) / S_increment

    # Vega computation
    new_sigma = sigma + 0.05
    ST_new = np.zeros(N)
    for i in range(N):
        S = S0
        for t in range(int(T * 365)):
            S += r * S * dt + new_sigma * S * np.sqrt(dt) * z[i]
            S = max(S, 0)  # Ensure stock price doesn't go negative
        ST_new[i] = S
    put_price_new = np.exp(-r * T) * np.mean(np.maximum(K - ST_new, 0))
    vega_put = (put_price_new - put_price) / 0.05

    return put_price, delta_put, vega_put


S0 = 100
K = 100
r = 0.05
sigma = 0.20
T = 0.25
N = 100000

# Perform Monte Carlo simulation for American put option
put_price, delta_put, vega_put = monte_carlo_simulation_american_put(S0, K, r, sigma, T, N)


print("American Put Option Price:", round(put_price, 2))
print("Delta for American Put Option:", round(delta_put, 2))
print("Vega for American Put Option:", round(vega_put, 2))

American Put Option Price: 22.54
Delta for American Put Option: -0.49
Vega for American Put Option: 61.63


Q6: Team member C

In [ ]:
import numpy as np
from tabulate import tabulate

In [ ]:
import numpy as np
import pandas as pd

def monte_carlo_simulation_american_call_and_put(S0, r, sigma, T, N, strike_prices):
    results = pd.DataFrame(index=strike_prices, columns=['Call Price', 'Call_Delta', 'Call_Vega', 'Put Price', 'Put_Delta', 'PutVega'])

    for K in strike_prices:
        # Monte Carlo simulation for American call option
        call_price, delta_call, vega_call = monte_carlo_simulation_american_call(S0, K, r, sigma, T, N)

        # Monte Carlo simulation for American put option
        put_price, delta_put, vega_put = monte_carlo_simulation_american_put(S0, K, r, sigma, T, N)

        # Update the results DataFrame
        results.loc[K] = [round(call_price, 2), round(delta_call, 2), round(vega_call, 2),
                           round(put_price, 2), round(delta_put, 2), round(vega_put, 2)]

    results.index.name = 'Strike Price'  # Set the header for the index column
    return results

# Parameters
S0 = 100
r = 0.05
sigma = 0.20
T = 0.25
N = 100000
strike_prices = [90, 95, 100, 105, 110]

# Perform Monte Carlo simulation for American call and put options
results = monte_carlo_simulation_american_call_and_put(S0, r, sigma, T, N, strike_prices)

# Display the results in pandas table
print(results)


             Call Price Delta (Call) Vega (Call) Put Price Delta (Put)  \
Strike Price                                                             
90                86.03         0.54      950.74     18.13       -0.45   
95                82.32         0.52      931.72     20.21       -0.47   
100                80.0          0.5      937.33     22.69       -0.49   
105               77.65         0.48      937.31     25.12       -0.51   
110                75.2         0.46      931.64     27.59       -0.53   

             Vega (Put)  
Strike Price             
90                61.23  
95                61.26  
100                61.8  
105               61.59  
110               60.94  


# Step 3
### Q7 : Team Member A

In [ ]:
import numpy as np
import scipy.stats as ss

# Black-Scholes Formula Implementation
def black_scholes(S, K, T, r, vol, option_type="C"):
    # Compute d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)

    if option_type == "C":
        # Call option
        Opt_Price = S * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
        Delta = ss.norm.cdf(d1)
    else:
        # Put option
        Opt_Price = K * np.exp(-r * T) * ss.norm.cdf(-d2) - S * ss.norm.cdf(-d1)
        Delta = -ss.norm.cdf(-d1)

    return round(Opt_Price, 2), round(Delta, 2)

# Given data
T = 0.25  # supposed in years. It is not the maturity, but the time to maturity
S = 100.0  # current price of the asset
r = 0.05   # risk-free rate
vol = 0.20  # volatility

# Pricing European Call option with 110% moneyness
K_call_110 = S / 1.10  # strike price for 110% moneyness
call_110_price, call_110_delta = black_scholes(S, K_call_110, T, r, vol, "C")

# Pricing European Put option with 95% moneyness
K_put_95 = S / 0.95  # strike price for 95% moneyness
put_95_price, put_95_delta = black_scholes(S, K_put_95, T, r, vol, "P")

# Portfolio 1: Buying both the call and put
portfolio_1_delta = round(call_110_delta + put_95_delta, 2)  # Sum of deltas

# Portfolio 2: Buying the call and selling the put
portfolio_2_delta = round(call_110_delta - put_95_delta, 2)  # Net delta

(call_110_price, call_110_delta, put_95_price, put_95_delta, portfolio_1_delta, portfolio_2_delta)


(10.9, 0.87, 6.35, -0.63, 0.24, 1.5)

Call option with 110% moneyness price: $10.9

Call option delta: 0.87

Put option with 95% moneyness price: $6.35

Put option delta: -0.63

Portfolio 1 delta (buying both the call and put): 0.24

Portfolio 2 delta (buying the call and selling the put): 1.5

These values suggest the following insights:

Call Option (110% moneyness): The price of the call option is $10.9, and the delta of 0.87 indicates that for every dollar increase in the price of the underlying asset, the price of the call option is expected to increase by 87 cents.

Put Option (95% moneyness): The put option is priced at $6.35. A negative delta of -0.63 implies that for every dollar increase in the price of the underlying asset, the price of the put option is expected to decrease by 63 cents.

Portfolio 1 Delta: By purchasing both the call and put option, the portfolio delta is 0.24. This low delta value indicates that the portfolio is less sensitive to changes in the price of the underlying asset. It suggests a nearly delta-neutral position, which might be part of a hedging strategy that benefits from changes in other factors like volatility rather than price movements.

Portfolio 2 Delta: By buying the call and selling the put, the portfolio has a delta of 1.5. This is a leveraged position that would benefit from an upward movement in the price of the underlying asset. For every dollar increase in the price of the underlying asset, the portfolio's value would increase by $1.5.

Q8 : Team member B

In [ ]:
import numpy as np

def monte_carlo_simulation_uao_barrier_option(S0, K, barrier, r, sigma, T, N):
    dt = 1 / 365
    ST = np.zeros(N)

    z = np.random.normal(0, 1, (N, int(T * 365)))

    for i in range(N):
        S = S0
        for t in range(int(T * 365)):
            S += r * S * dt + sigma * S * np.sqrt(dt) * z[i, t]
            if S >= barrier:
                S = 0
            ST[i] = S


    option_payoffs = np.maximum(0, ST - K)  #option value at maturity


    option_price = np.exp(-r * T) * np.mean(option_payoffs)

    return option_price


S0 = 120
K = 120
barrier = 141
r = 0.06
sigma = 0.30
T = 8 / 12
N = 100000

option_price = monte_carlo_simulation_uao_barrier_option(S0, K, barrier, r, sigma, T, N)

#Output
print("Estimated price of the Up-and-Out barrier option:", round(option_price, 2))


Estimated price of the Up-and-Out barrier option: 0.67


Q9: Team member C

In [ ]:
import numpy as np

def monte_carlo_simulation_uai_barrier_option(S0, K, barrier, r, sigma, T, N):
    dt = 1 / 365
    ST = np.zeros(N)

    z = np.random.normal(0, 1, (N, int(T * 365)))

    for i in range(N):
        S = S0
        barrier_crossed = False
        for t in range(int(T * 365)):
            S += r * S * dt + sigma * S * np.sqrt(dt) * z[i, t]
            if not barrier_crossed and S >= barrier:
                barrier_crossed = True
            if barrier_crossed:
                S = max(S, 0)  # Ensure stock price doesn't go negative
            ST[i] = S

    option_payoffs = np.maximum(0, ST - K)  # Option value at maturity

    option_price = np.exp(-r * T) * np.mean(option_payoffs)

    return option_price

# Parameters
S0 = 120
K = 120
barrier = 110  # Up-and-In barrier level
r = 0.06
sigma = 0.30
T = 8 / 12
N = 100000

option_price = monte_carlo_simulation_uai_barrier_option(S0, K, barrier, r, sigma, T, N)

# Output
print("Estimated price of the Up-and-In barrier option:", round(option_price, 2))


Estimated price of the Up-and-In barrier option: 14.01


In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes_vanilla_option(S0, K, T, r, sigma, option_type='call'): #Defining the function for computing the price of vanilla option

    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2) #Option price for a call option
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1) #Option price for a put option
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

    return option_price

# Estimated price of vanilla call option
vanilla_call_price = black_scholes_vanilla_option(S0, K, T, r, sigma, option_type='call')
print("Estimated price of the vanilla call option:", round(vanilla_call_price, 2))

# Estimated price of vanilla put option
vanilla_put_price = black_scholes_vanilla_option(S0, K, T, r, sigma, option_type='put')
print("Estimated price of the vanilla put option:", round(vanilla_put_price, 2))



Estimated price of the vanilla call option: 13.97
Estimated price of the vanilla put option: 9.27
